In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os

In [6]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb", "depth", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 1
        }

dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

217


In [7]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=4, sampler=weighted_sampler, collate_fn=None)

In [8]:
dataset.data_shapes

array([[  1, 160, 960,   3],
       [  1, 160, 960,   3],
       [  0,   0,   3,   1]])

In [9]:
dataset.used_inputs[0]

'rgb'

In [10]:
# Weird: When num_workers increases (>0) then it takes longer to loop over batches??
count = 0
for batch in dl:
    count += 1
print(count)

14


In [9]:
dataset.data_shapes

array([[  1, 160, 960,   3],
       [  1, 160, 960,   3],
       [  0,   0,   3,   1]])

In [22]:
batch["depth"].shape

torch.Size([9, 5, 160, 960, 3])